In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env
import polars as pl

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)

In [ ]:

from humbldata.toolbox.technical.mandelbrot_channel.model import (
    calc_mandelbrot_channel,
    calc_mandelbrot_channel_historical,
)


data = (
    obb.equity.price.historical(
        "AAA",
        provider="yfinance",
        start_date="2020-07-01",
        end_date="2020-07-10",
        adjustment="splits_and_dividends",
    ).to_polars()
).drop(["dividends", "stock_splits"])

# data = obb.etf.historical(["ETHU"],start_date="2020-01-01", end_date="2024-07-01", provider="yfinance")
# data.to_polars()

# mandelbrot = calc_mandelbrot_channel(  # noqa: ERA001
#         data,
#         window="1m",
#         rv_adjustment=True,
#         rv_method="std",
#         rv_grouped_mean=False,
#         rs_method="RS",
#         live_price=False,
#     ).collect()
# mandelbrot

In [ ]:
data

## Test `calc_up_down_pct`


In [ ]:
from humbldata.portfolio.analytics.user_table.helpers import calc_up_down_pct


df = pl.DataFrame({
    "symbol": ["AAPL", "GOOGL", "MSFT"],
    "bottom_price": [5.18, 15.07, 16.24],
    "recent_price": [10.05, 20.31, 16.42],
    "top_price": [11.23, 25.17, 30.09],
})
result = calc_up_down_pct(df)
print(result)

# Setting up UserTable

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.standard_models.portfolio.analytics.user_table import UserTableQueryParams
from humbldata.core.utils.openbb_helpers import get_latest_price


symbols = UserTableQueryParams(symbol="AAPL, MSFT, NVDA").symbol
symbols

# Tesing Async


## Get Sector Async

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_category, aget_equity_sector
import polars as pl

stocks = pl.Series([
    "GOOGL",  # Communication Services
    "AMZN",   # Consumer Discretionary
    "KO",     # Consumer Staples
    "XOM",    # Energy
    "JPM",    # Financials
    "JNJ",    # Health Care
    "HON",    # Industrials
    "AAPL",   # Information Technology
    "LIN",    # Materials
    "PLD",    # Real Estate
    "NEE"     # Utilities
])

# Define objects and lists for different symbol groups

# Equities
equity_symbols = ["AAPL", "NVDA", "TSLA"]

# ETFs
etf_symbols = ["XLE", "XLF", "XLU"]

# Mixed (equities and ETFs)
mixed_symbols = ["AAPL", "XLE", "XLU", "TSLA"]

# Edge cases
edge_symbols = ["BITI", "LNGG", "ETHU", "DBA"]


# test = (await aget_etf_category(symbols=["AAPL", "XLE", "DBA"], provider="yfinance")).collect()
test2 = (await aget_equity_sector(symbols=equity_symbols, provider="yfinance")).collect()
# test2 = (await aget_equity_sector(symbols=["XLE", "AAPL"], provider="yfinance")).collect()
test2

In [ ]:
symbols = ["CORN", 'SLV', 'DBA', "XLE", "AAPL", "FXE", "SPY", "QQQ", "BITO", "MAXI", "SATO", "BIL", "LQD", "QAI", "MNA", "CLSE"]
symbols2 = ["AAPL", "FXE", "SPY", "MAXI", "GLD"]
symbols3 = commodity_etf_symbols = ["GLD", "IAU", "SLV", "GLDM", "PDBC", "SGOL", "FTGC", "DBC", "SIVR", "USO", "IAUM", "GSG", "BCI", "PPLT", "COMT", "GLTR", "OUNZ", "BAR", "UNG", "DBA", "AAAU", "DJP", "CMDT", "KRBN", "PALL", "COM", "NBCM", "CMDY", "DBO", "BCD", "KCCA", "CPER", "HGER", "USCI", "DBB", "DBP", "GCC", "WEAT", "BNO", "UGA", "DGP", "IGLD", "COMB", "DBE", "FGDL", "CORN", "USL", "CCRV", "DJCB", "IAUF", "PLTM", "BDRY", "PDBA", "BGLD", "UCIB", "SOYB", "GRN", "PIT", "BCIM", "RENW", "SDCI", "UNL", "SHNY", "DCMT", "TAGS", "CANE", "TMET", "KEUA", "HCOM", "HARD", "EVMT", "USG", "AMPD", "DZZ", "DGZ", "KMET", "CMCI", "USOY", "ZSC", "TILL", "BWET", "DULL", "ZSB", "LNGG", "USOI", "OILK", "SLVO", "GLDI", "BOIL", "KOLD", "AGQ", "GLL", "UGL", "ZSL", "UCO", "SCO"]



df = (await aget_etf_category(symbols=edge_symbols)).collect()
df

In [ ]:
from humbldata.portfolio.analytics.user_table.helpers import normalize_asset_class


normalized_df = normalize_asset_class(df).select(["symbol", "category"])
normalized_df

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_category


df2 = (await aget_etf_category(symbols=edge_symbols)).collect()
df2

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_asset_class_filter

(await aget_asset_class_filter(symbols=["XLE", "XLF", "XLK"])).collect()

## Test aget_sector_filter

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_sector_filter

etf_data = pl.LazyFrame({
    "symbol": ["BITI", "LNGG", "ETHU", "DBA"],
    "category": [
        "Trading--Miscellaneous",
        "Equity Energy",
        None,
        "Commodities Focused",
    ]
})

(await aget_sector_filter(symbols=mixed_symbols)).collect()

# Test user_table_engine

In [ ]:
from humbldata.portfolio.analytics.user_table.model import user_table_engine
import polars as pl

etf_data = [
    pl.DataFrame(
        {"symbol": ["AAPL", "NVDA", "TSLA"], "category": [None, None, None]}
    ),
    pl.DataFrame(
        {
            "symbol": ["XLE", "XLF", "XLU"],
            "category": ["Equity Energy", "Financial", "Utilities"],
        }
    ),
    pl.DataFrame(
        {
            "symbol": ["AAPL", "XLE", "XLU", "TSLA"],
            "category": [None, "Equity Energy", "Utilities", None],
        }
    ),
    pl.DataFrame(
        {
            "symbol": ["BITI", "LNGG", "ETHU", "DBA"],
            "category": [
                "Trading--Miscellaneous",
                "Equity Energy",
                None,
                "Commodities Focused",
            ],
        }
    ),
]

out: pl.LazyFrame = await user_table_engine(symbols=edge_symbols)
out.collect()

## Test UserTableFetcher

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.portfolio_controller import Portfolio

portfolio = Portfolio(symbols=["AAPL"], user_role="anonymous")
portfolio

In [ ]:
result = (await portfolio.analytics.user_table())

In [ ]:
result.to_polars()

In [ ]:
result.to_polars()

# Testing Sandbox

In [1]:
import polars as pl

In [3]:
pl.read_parquet("../tests/test_data/test_data.parquet").filter(
        pl.col("symbol") == "AAPL"
    )

date,open,high,low,close,volume,split_ratio,dividend,symbol,log_returns,window_index,window_mean,detrended_log_returns,cum_sum,cum_sum_min,cum_sum_max,cum_sum_range,cum_sum_std,realized_volatility,vol_bucket
date,f64,f64,f64,f64,f64,f64,f64,str,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,str
2000-01-05,0.784238,0.835733,0.778569,0.786128,7.783216e8,0.0,0.0,"""AAPL""",0.014527,287,-0.00537,0.019896,-0.062811,-0.212293,0.08333,0.295623,0.091599,21.971445,"""low"""
2000-01-06,0.802191,0.808805,0.718098,0.718098,7.679728e8,0.0,0.0,"""AAPL""",-0.090514,287,-0.00537,-0.085144,-0.147955,-0.212293,0.08333,0.295623,0.091599,26.168462,"""mid"""
2000-01-07,0.729436,0.763452,0.721878,0.752113,4.607344e8,0.0,0.0,"""AAPL""",0.046281,287,-0.00537,0.051651,-0.096305,-0.212293,0.08333,0.295623,0.091599,28.475358,"""mid"""
2000-01-10,0.77101,0.7729,0.716208,0.738885,5.05064e8,0.0,0.0,"""AAPL""",-0.017744,287,-0.00537,-0.012375,-0.108679,-0.212293,0.08333,0.295623,0.091599,32.249623,"""mid"""
2000-01-11,0.725184,0.751168,0.684083,0.70109,4.415488e8,0.0,0.0,"""AAPL""",-0.052506,287,-0.00537,-0.047137,-0.155816,-0.212293,0.08333,0.295623,0.091599,37.15822,"""mid"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-12-22,194.93126,195.160978,192.724085,193.353287,3.71228e7,0.0,0.0,"""AAPL""",-0.005563,0,0.000788,-0.006351,0.008694,-0.002047,0.036482,0.038529,0.011751,15.926591,"""low"""
2023-12-26,193.36327,193.642911,192.584265,192.803986,2.89193e7,0.0,0.0,"""AAPL""",-0.002845,0,0.000788,-0.003633,0.005061,-0.002047,0.036482,0.038529,0.011751,15.769927,"""low"""
2023-12-27,192.244692,193.253399,190.846467,192.903839,4.80877e7,0.0,0.0,"""AAPL""",0.000518,0,0.000788,-0.00027,0.004791,-0.002047,0.036482,0.038529,0.011751,15.419317,"""low"""
